#Part 5: fit GLM and use as input to HDDM

In this last part we will fit a GLM to the fMRI data and use the parameters we estimate to link the neural data to the drift diffusion model

In [339]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import scipy as sp
from scipy import stats
%matplotlib inline

### Get the signals

We use a glob and regular experssion to get the data:

In [340]:
import glob
import re

In [341]:
fns = glob.glob('/data/extracted_signals/extracted_timeseries/_mask_*_subject_id_*/_extracter*/*.txt')
fns[:10]

['/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0483/_extracter0/pp0483_B1_dtype_mcf_mask_gms_tempfilt_warp_ts.txt',
 '/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0483/_extracter1/pp0483_B2_dtype_mcf_mask_gms_tempfilt_warp_ts.txt',
 '/data/extracted_signals/extracted_timeseries/_mask_STR_R_subject_id_0553/_extracter0/pp0553_B1_dtype_mcf_mask_gms_tempfilt_warp_ts.txt',
 '/data/extracted_signals/extracted_timeseries/_mask_STR_R_subject_id_0553/_extracter1/pp0553_B2_dtype_mcf_mask_gms_tempfilt_warp_ts.txt',
 '/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0197/_extracter0/pp0197_B1_dtype_mcf_mask_gms_tempfilt_warp_ts.txt',
 '/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0197/_extracter1/pp0197_B2_dtype_mcf_mask_gms_tempfilt_warp_ts.txt',
 '/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0551/_extracter0/pp0551_B1_dtype_mcf_mask_gms_tempfilt_warp_ts.txt',
 '/data/extracted_signals/e

1) What does the following code do?

In [342]:
reg = re.compile('.*/_mask_(?P<mask>.*)_subject_id_(?P<subj_idx>.*)/_extracter[0-9]/.*_B(?P<block>[0-9])_dtype_mcf_mask_gms_tempfilt_warp_ts.txt')

In [343]:
for fn in fns[:5]:
    print fn, reg.match(fn).groupdict()

/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0483/_extracter0/pp0483_B1_dtype_mcf_mask_gms_tempfilt_warp_ts.txt {'subj_idx': '0483', 'mask': 'STR_L', 'block': '1'}
/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0483/_extracter1/pp0483_B2_dtype_mcf_mask_gms_tempfilt_warp_ts.txt {'subj_idx': '0483', 'mask': 'STR_L', 'block': '2'}
/data/extracted_signals/extracted_timeseries/_mask_STR_R_subject_id_0553/_extracter0/pp0553_B1_dtype_mcf_mask_gms_tempfilt_warp_ts.txt {'subj_idx': '0553', 'mask': 'STR_R', 'block': '1'}
/data/extracted_signals/extracted_timeseries/_mask_STR_R_subject_id_0553/_extracter1/pp0553_B2_dtype_mcf_mask_gms_tempfilt_warp_ts.txt {'subj_idx': '0553', 'mask': 'STR_R', 'block': '2'}
/data/extracted_signals/extracted_timeseries/_mask_STR_L_subject_id_0197/_extracter0/pp0197_B1_dtype_mcf_mask_gms_tempfilt_warp_ts.txt {'subj_idx': '0197', 'mask': 'STR_L', 'block': '1'}


Let's put everything in a large dataframe:

In [344]:
df = []

for fn in fns:
    d = reg.match(fn).groupdict()
    d['signal'] = np.loadtxt(fn)
    df.append(d)
    
df = pandas.DataFrame(df)
df['block'] = df['block'].astype(int)
df['subj_idx'] = df['subj_idx'].astype(int)

2) Have a look at the head of the dataframe, what do you see? Can you plot the signal for a few subjects?

Now we are gonna use nipy to set up a design matrix for the GLM

In [348]:
from nipy.modalities.fmri import design_matrix, experimental_paradigm


Let's start with an example:

In [349]:
conditions = ['condition1'] * 3 + ['condition2'] * 3
onsets = [5, 25 , 35, 15, 30, 40]


tr = 2.0
frametimes = np.arange(0, 50, 2)
hrf_model = 'Canonical'

paradigm =  experimental_paradigm.BlockParadigm(con_id=conditions, 
                                                onset=onsets,
                                                duration=[[1.]] * len(conditions))


X, names= design_matrix.dmtx_light(frametimes, paradigm, hrf_model=hrf_model, )

1) Plot matrix X. What do you see? What are the 'conditions' and 'onsets'?

Now let's make the GLM for an actual subject

In [351]:
behavior = pandas.load('/data/behavior-v2.pandas')

Let's go for subject 548, block 1, left Striatum

In [352]:
subj_idx = 548
block = 1
mask = 'STR_L'

# Select only onsets of subject subj_idx and block block
d = behavior[(behavior.subj_idx == subj_idx) & (behavior.block == block)]

# Make a list of conditions
conditions = ['accuracy'] * (d.cond == '1').sum() +  ['speed'] * (d.cond == '2').sum()

# Make a list of onsets
onsets = d[d.cond == '1'].cue_onset.tolist() + d[d.cond == '2'].cue_onset.tolist()


# The number of volumes is 355, the time is 355*2 = 710, in steps of TR seconds (2)
tr = 2.0
frametimes = np.arange(0, 710, tr)
hrf_model = 'Canonical'

# Set up paradigm
paradigm =  experimental_paradigm.BlockParadigm(con_id=conditions, 
                                                onset=onsets,
                                                duration=[[2.]] * len(conditions))

# Set up GLM
X, names= design_matrix.dmtx_light(frametimes, paradigm, hrf_model=hrf_model, drift_model='blank')
X = pandas.DataFrame(X, columns=names)

2) Plot X again, how does it look?

Now we can fit the GLM using ordinary least squares (OLS)

In [353]:
import statsmodels.api as sm
Y = df[(df.subj_idx == subj_idx) & (df.block == block) & (df['mask'] == mask)].iloc[0].signal
model = sm.OLS(Y, X)
r = model.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.127
Method:                 Least Squares   F-statistic:                     26.75
Date:                Tue, 02 Jun 2015   Prob (F-statistic):           1.53e-11
Time:                        09:39:17   Log-Likelihood:                -1623.5
No. Observations:                 355   AIC:                             3253.
Df Residuals:                     352   BIC:                             3265.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
accuracy      19.9825      8.632      2.315      0.021         3.006    36.959
speed         61.2510      8.432      7.264      0.000        44.668    77.834
constant      -4.5229      1.445     -3.129      0.002        -7.365    -1.680
==============================================================================
Omnibus:                        1.475   Durbin-Watson:                   1.088
Prob(Omnibus):                  0.478   Jarque-Bera (JB):                1.225
Skew:                           0.119   Prob(JB):                        0.542
Kurtosis:                       3.162   Cond. No.                         7.54
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3) Interpret these resuts. What do you see?

4) Now write a [function](http://www.tutorialspoint.com/python/python_functions.htm) that, given a subject_id and a block fits the glm and returns a dataframe with parameter estimates (hint: most of this is just copying the code above).

In [355]:
def fit_glm(subj_idx, block, mask):
    
    ....
    
    r = model.fit()
    
    return r.params
    

In [356]:

fit_glm(548, 1, 'STR_R')

accuracy     9.676504
speed       45.783643
constant    -3.099225
dtype: float64

Now we can loop through the signals and fit the GLM, storing it in a dataframe

In [357]:
results = []

for fn in fns:
    subj_idx = int(reg.match(fn).groupdict()['subj_idx'])
    block = int(reg.match(fn).groupdict()['block'])
    for mask in ['STR_L', 'STR_R']:
        r = fit_glm(subj_idx, block, mask)
        
        for condition in ['accuracy', 'speed']:
            d = {}
            d['subj_idx'] = subj_idx
            d['mask'] = mask
            d['block'] = block
            d['condition'] = condition
            d['value'] = r[condition]
            results.append(d)

results = pandas.DataFrame(results)

5) Have a look at the dataframe 'results' and use factorplot to summarize it

6) Interpret the results

Now we can use pivot_table to get a difference-value for every subjects

In [ ]:
neural_results = results.pivot_table(values='value', columns=['condition', 'mask'], index=['subj_idx'])
neural_results

7) What happened here?

Now let's load the hddm parameters

In [362]:
hddm_pars = pandas.load('/data/results_hddm.pandas')
hddm_pars.head(15)

The following code is a bit hacky, a way to get the threshold-estimates out

In [364]:
import re
reg = re.compile('a_subj\((?P<condition>[0-9])\)\.(?P<subj_idx>[0-9]{3})')

results = []

for i, row in hddm_pars.iterrows():
    if row.parameter[:6] == 'a_subj':
        d = reg.match(row.parameter).groupdict()
        d['value'] = row.value
        results.append(d)
        
threshold_pars = pandas.DataFrame(results)
threshold_pars['condition'] = threshold_pars.condition.map({'1':'accuracy', '2':'speed'})
threshold_pars['subj_idx'] = threshold_pars['subj_idx'].astype(int) 

In [ ]:
behavioral_results = threshold_pars.pivot_table(values='value', columns='condition', index=['subj_idx'])
behavioral_results

Now we merge the behavioral and neural results

In [ ]:
behavioral_results = threshold_pars.pivot_table(values='value', columns='condition', index=['subj_idx'])
behavioral_results

8) What does the table above contain?

9) Correlate the difference in thresholds with the difference in fMRI signals using lmplot for left and right striatum